In [1]:
import read

In [2]:
read.query_data_range(years=2020, pollutants=['SO2', 'CO', 'NO2'], expositions=1, regions=['MALOPOLSKIE', 'MAZOWIECKIE', 'SLASKIE'])

,timestamp,station_code,measurement,pollutant,exposition,region
0,2020-01-01 01:00:00,MpKrakAlKras,0.67896,CO,1,MALOPOLSKIE
1,2020-01-01 02:00:00,MpKrakAlKras,0.72196,CO,1,MALOPOLSKIE
2,2020-01-01 03:00:00,MpKrakAlKras,0.75515,CO,1,MALOPOLSKIE
3,2020-01-01 04:00:00,MpKrakAlKras,0.73271,CO,1,MALOPOLSKIE
4,2020-01-01 05:00:00,MpKrakAlKras,0.66290,CO,1,MALOPOLSKIE
...,...,...,...,...,...,...
825691,2020-12-31 20:00:00,SlZywieKoper,27.43300,NO2,1,SLASKIE
825692,2020-12-31 21:00:00,SlZywieKoper,14.54020,NO2,1,SLASKIE
825693,2020-12-31 22:00:00,SlZywieKoper,12.71480,NO2,1,SLASKIE
825694,2020-12-31 23:00:00,SlZywieKoper,11.78310,NO2,1,SLASKIE
